In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/check-attack/attack_syn_flood (1).csv
/kaggle/input/check-attack/attack_portscan (2).csv
/kaggle/input/network-intrusion-dataset/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv
/kaggle/input/network-intrusion-dataset/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv
/kaggle/input/network-intrusion-dataset/Tuesday-WorkingHours.pcap_ISCX.csv
/kaggle/input/network-intrusion-dataset/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv
/kaggle/input/network-intrusion-dataset/Monday-WorkingHours.pcap_ISCX.csv
/kaggle/input/network-intrusion-dataset/Friday-WorkingHours-Morning.pcap_ISCX.csv
/kaggle/input/network-intrusion-dataset/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv
/kaggle/input/network-intrusion-dataset/Wednesday-workingHours.pcap_ISCX.csv
/kaggle/input/multiclass-ids/keras/default/1/best_model_multiclass.keras


In [2]:

import os, json, time, gc, warnings, random
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, RobustScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                             f1_score, confusion_matrix, classification_report)

import joblib
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import (Conv1D, MaxPooling1D, BatchNormalization,
                                     SpatialDropout1D, Dropout, Dense, LSTM,
                                     Bidirectional, GlobalAveragePooling1D,
                                     LayerNormalization)
from tensorflow.keras.layers import MultiHeadAttention, Add
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

# reproducibility
SEED = 42

2026-02-06 18:03:20.357187: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770401000.739523      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770401000.910352      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770401001.872442      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770401001.872491      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770401001.872494      55 computation_placer.cc:177] computation placer alr

In [3]:
paths = [
    "/kaggle/input/network-intrusion-dataset/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv",
    "/kaggle/input/network-intrusion-dataset/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv",
    "/kaggle/input/network-intrusion-dataset/Tuesday-WorkingHours.pcap_ISCX.csv",
    "/kaggle/input/network-intrusion-dataset/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv",
    "/kaggle/input/network-intrusion-dataset/Monday-WorkingHours.pcap_ISCX.csv",
    "/kaggle/input/network-intrusion-dataset/Friday-WorkingHours-Morning.pcap_ISCX.csv",
    "/kaggle/input/network-intrusion-dataset/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv",
    "/kaggle/input/network-intrusion-dataset/Wednesday-workingHours.pcap_ISCX.csv"
]



In [4]:
def load_concat(paths):
    dfs = []
    for p in paths:
        temp = pd.read_csv(p, low_memory=False)
        temp.columns = temp.columns.str.strip()
        dfs.append(temp)

    df = pd.concat(dfs, ignore_index=True, copy=False)
    return df

In [5]:
def preprocess(df):
    dfc = df.copy()
    dfc.columns = dfc.columns.str.strip()
    FEATURES = [
    "Flow Duration",
    "Total Fwd Packets",
    "Total Backward Packets",
    "Total Length of Fwd Packets",
    "Total Length of Bwd Packets",

    "Fwd Packet Length Min",
    "Fwd Packet Length Max",
    "Fwd Packet Length Mean",
    "Fwd Packet Length Std",

    "Bwd Packet Length Min",
    "Bwd Packet Length Max",
    "Bwd Packet Length Mean",
    "Bwd Packet Length Std",

    "Flow Bytes/s",
    "Flow Packets/s",

    "Flow IAT Mean",
    "Flow IAT Std",
    "Fwd IAT Mean",
    "Fwd IAT Std",
    "Bwd IAT Mean",
    "Bwd IAT Std",

    "Min Packet Length",
    "Max Packet Length",
    "Packet Length Mean",
    "Packet Length Std",
    "Packet Length Variance",

    "FIN Flag Count",
    "SYN Flag Count",
    "RST Flag Count",
    "PSH Flag Count",
    "ACK Flag Count",
    "URG Flag Count",
    "ECE Flag Count",

    "Fwd PSH Flags",
    "Bwd PSH Flags",
    "Fwd URG Flags",
    "Bwd URG Flags",

    "Down/Up Ratio",

    "Average Packet Size",
    "Avg Fwd Segment Size",
    "Avg Bwd Segment Size",

    "Active Mean",
    "Active Std",
    "Idle Mean",
    "Idle Std"
]

    LABEL = "Label"
    df_selected = dfc[FEATURES + [LABEL]]
    df_selected["Label"] = (df_selected["Label"].astype(str).str.strip().str.lower()
                    .replace({"normal": "benign"}))
    y = df_selected["Label"]
    X = df_selected.drop(columns=["Label"])
    X = X.apply(pd.to_numeric, errors="coerce")
    X.replace([np.inf, -np.inf], np.nan, inplace=True)
    X.fillna(0, inplace=True)
    
    df_clean = pd.concat([X, y], axis=1)
    return df_clean
    print("Final shape:", df_clean.shape)
    print("Columns kept:", df_clean.columns.tolist())



In [6]:
def normalize_label(s: str) -> str:
    s = str(s).lower().strip()
    s = (s.replace('web attack –', 'web attack -')
           .replace('web attack —', 'web attack -')
           .replace('web attack �', 'web attack -'))
    if s == 'benign':
        return 'benign'
    if 'web attack' in s or 'xss' in s or 'sql injection' in s:
        return 'webattack'
    if any(k in s for k in ['hulk', 'goldeneye', 'slowloris', 'slowhttptest']):
        return 'dos'
    if 'ddos' in s:
        return 'ddos'
    if any(k in s for k in ['ftp-patator', 'ssh-patator']):
        return 'bruteforce'
    if 'portscan' in s or 'port scan' in s:
        return 'portscan'
    if 'bot' in s:
        return 'bot'
    if 'infiltration' in s:
        return 'infiltration'
    return s


In [7]:
df_raw = load_concat(paths)
df = preprocess(df_raw)
df["Label"] = df["Label"].astype(str).map(normalize_label)
print("After clean:", df.shape)
print(df["Label"].value_counts())

After clean: (2830743, 46)
Label
benign          2273097
dos              252661
portscan         158930
ddos             128027
bruteforce        13835
webattack          2180
bot                1966
infiltration         36
heartbleed           11
Name: count, dtype: int64


In [12]:
ATTACK_CAP = 30000

df_attacks = df[df["Label"] != "benign"]

df_attacks = (
    df_attacks
    .groupby("Label", group_keys=False)
    .apply(lambda x: x.sample(
        n=min(len(x), ATTACK_CAP),
        random_state=42
    ))
)

print(df_attacks["Label"].value_counts())


Label
dos             30000
ddos            30000
portscan        30000
bruteforce      13835
webattack        2180
bot              1966
infiltration       36
heartbleed         11
Name: count, dtype: int64


In [13]:
import os

ART_DIR = "./artifacts"
os.makedirs(ART_DIR, exist_ok=True)


In [14]:
from sklearn.preprocessing import LabelEncoder
import joblib

le_attack = LabelEncoder()
df_attacks["attack_label"] = le_attack.fit_transform(df_attacks["Label"])

print("Final attack classes:", list(le_attack.classes_))
joblib.dump(le_attack, "attack_label_encoder.pkl")


Final attack classes: ['bot', 'bruteforce', 'ddos', 'dos', 'heartbleed', 'infiltration', 'portscan', 'webattack']


['attack_label_encoder.pkl']

In [15]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import PowerTransformer

In [16]:

X = df_attacks.drop(columns=["Label", "attack_label"])
y = df_attacks["attack_label"]   
 

X_train, X_test, y_train_idx, y_test_idx = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=SEED
)
print( X_train.shape, X_test.shape)
selector = VarianceThreshold(0.01)
X_train_sel = selector.fit_transform(X_train)
X_test_sel  = selector.transform(X_test)

scaler = PowerTransformer(method="yeo-johnson", standardize=True)
X_train_ml = scaler.fit_transform(X_train_sel)
X_test_ml  = scaler.transform(X_test_sel)
num_classes = len(np.unique(y_train_idx))
y_train_cnn = y_train_idx.values
y_test_cnn  = y_test_idx.values

X_train_cnn = X_train_ml[..., np.newaxis]
X_test_cnn  = X_test_ml[...,  np.newaxis]

cls = np.unique(y_train_idx)
cw_values = compute_class_weight(class_weight="balanced", classes=cls, y=y_train_idx)
class_weight = {int(c): float(w) for c, w in zip(cls, cw_values)}
print({"class_weight": class_weight})


joblib.dump(selector, os.path.join(ART_DIR, "selector.pkl"))
joblib.dump(scaler,   os.path.join(ART_DIR, "scaler.pkl"))


print(f"ML 2D  -> X_train: {X_train_ml.shape}, X_test: {X_test_ml.shape}")
print(f"CNN 3D -> X_train: {X_train_cnn.shape}, X_test: {X_test_cnn.shape}")
print(f"y_train (idx/oh): {y_train_idx.shape} / {y_train_cnn.shape}")

np.savez_compressed(
    os.path.join(ART_DIR, "prepared_train_test.npz"),
    X_train_ml=X_train_ml, X_test_ml=X_test_ml,
    y_train_idx=y_train_idx, y_test_idx=y_test_idx,
    X_train_cnn=X_train_cnn, X_test_cnn=X_test_cnn,
    y_train_cnn=y_train_cnn, y_test_cnn=y_test_cnn,
)


(86422, 45) (21606, 45)
{'class_weight': {0: 6.871978371501272, 1: 0.9760345139139862, 2: 0.4501145833333333, 3: 0.4501145833333333, 4: 1200.3055555555557, 5: 372.5086206896552, 6: 0.4501145833333333, 7: 6.194237385321101}}
ML 2D  -> X_train: (86422, 40), X_test: (21606, 40)
CNN 3D -> X_train: (86422, 40, 1), X_test: (21606, 40, 1)
y_train (idx/oh): (86422,) / (86422,)


In [18]:
from tensorflow.keras.layers import (
    Input, Conv1D, BatchNormalization, SpatialDropout1D,
    MaxPooling1D, Bidirectional, LSTM, MultiHeadAttention,
    Add, LayerNormalization, GlobalAveragePooling1D,
    Dense, Dropout
)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

timesteps = X_train_cnn.shape[1]
channels  = X_train_cnn.shape[2]

inputs = Input(shape=(timesteps, channels), name="inputs")
x = Conv1D(128, 3, padding="same", activation="relu",
           kernel_regularizer=l2(1e-4))(inputs)
x = BatchNormalization()(x)
x = SpatialDropout1D(0.2)(x)
x = MaxPooling1D(2, padding="same")(x)

x = Conv1D(256, 3, padding="same", activation="relu",
           kernel_regularizer=l2(1e-4))(x)
x = BatchNormalization()(x)
x = SpatialDropout1D(0.2)(x)
x = MaxPooling1D(2, padding="same")(x)
x = Bidirectional(LSTM(128, return_sequences=True), name="bilstm")(x)

attn_out = MultiHeadAttention(
    num_heads=4,
    key_dim=64,
    name="mha"
)(x, x)

x = Add()([x, attn_out])
x = LayerNormalization()(x)
x = GlobalAveragePooling1D(name="gap")(x)

x = Dense(128, activation="relu", kernel_regularizer=l2(1e-4))(x)
x = Dropout(0.4)(x)

outputs = Dense(num_classes, activation="softmax", name="output")(x)

attn_model = Model(inputs, outputs, name="CNN_BiLSTM_Attn_multiclass")


I0000 00:00:1770401357.344064      55 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13757 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1770401357.350134      55 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13757 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


In [20]:
attn_model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

attn_model.summary()

Model: "CNN_BiLSTM_Attn_multiclass"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ inputs (InputLayer) │ (None, 40, 1)     │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 40, 128)   │        512 │ inputs[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 40, 128)   │        512 │ conv1d[0][0]      │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ spatial_dropout1d   │ (None, 40, 128)   │          0 │ batch_normalizat… │
│ (SpatialDropout1D)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d       │ (None, 20, 128)   │          0 │ spatial_dropout1… │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 20, 256)   │     98,560 │ max_pooling1d[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 20, 256)   │      1,024 │ conv1d_1[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ spatial_dropout1d_1 │ (None, 20, 256)   │          0 │ batch_normalizat… │
│ (SpatialDropout1D)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_1     │ (None, 10, 256)   │          0 │ spatial_dropout1… │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bilstm              │ (None, 10, 256)   │    394,240 │ max_pooling1d_1[… │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mha                 │ (None, 10, 256)   │    263,168 │ bilstm[0][0],     │
│ (MultiHeadAttentio… │                   │            │ bilstm[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 10, 256)   │          0 │ bilstm[0][0],     │
│                     │                   │            │ mha[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 10, 256)   │        512 │ add[0][0]         │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gap                 │ (None, 256)       │          0 │ layer_normalizat… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │     32,896 │ gap[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 128)       │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output (Dense)      │ (None, 8)         │      1,032 │ dropout_1[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 792,456 (3.02 MB)

 Trainable params: 791,688 (3.02 MB)

 Non-trainable params: 768 (3.00 KB)

In [21]:
batch_size = 256

train_dataset = tf.data.Dataset.from_tensor_slices((X_train_cnn, y_train_cnn))
train_dataset = train_dataset.shuffle(len(X_train_cnn), seed=SEED)
train_dataset = train_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

val_dataset = tf.data.Dataset.from_tensor_slices((X_test_cnn, y_test_cnn))
val_dataset = val_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)


xb, yb = next(iter(train_dataset))
print("Sanity shapes:", xb.shape, yb.shape)
print("One forward:", attn_model(xb, training=False).shape)

Sanity shapes: (256, 40, 1) (256,)


I0000 00:00:1770401472.609654      55 cuda_dnn.cc:529] Loaded cuDNN version 91002


One forward: (256, 8)


In [22]:
from tensorflow.keras.callbacks import (
    EarlyStopping,
    ReduceLROnPlateau,
    ModelCheckpoint
)
from sklearn.metrics import (
    classification_report,
    confusion_matrix
)
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os


In [23]:
CKPT_PATH = os.path.join(ART_DIR, "best_model_multiclass.keras")

callbacks = [
    EarlyStopping(
        monitor="val_loss",
        patience=6,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.5,
        patience=3,
        min_lr=1e-6,
        verbose=1
    ),
    ModelCheckpoint(
        CKPT_PATH,
        monitor="val_loss",
        save_best_only=True,
        verbose=1
    )
]


In [24]:
EPOCHS = 30

history = attn_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=EPOCHS,
    callbacks=callbacks,
    class_weight=class_weight,
    verbose=1
)


Epoch 1/30
338/338 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.4267 - loss: 1.8024
Epoch 1: val_loss improved from inf to 1.82950, saving model to ./artifacts/best_model_multiclass.keras
338/338 ━━━━━━━━━━━━━━━━━━━━ 17s 27ms/step - accuracy: 0.4271 - loss: 1.8012 - val_accuracy: 0.4383 - val_loss: 1.8295 - learning_rate: 1.0000e-04
Epoch 2/30
336/338 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.6802 - loss: 0.8772
Epoch 2: val_loss improved from 1.82950 to 0.30283, saving model to ./artifacts/best_model_multiclass.keras
338/338 ━━━━━━━━━━━━━━━━━━━━ 9s 25ms/step - accuracy: 0.6807 - loss: 0.8765 - val_accuracy: 0.9015 - val_loss: 0.3028 - learning_rate: 1.0000e-04
Epoch 3/30
336/338 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.8471 - loss: 0.5358
Epoch 3: val_loss improved from 0.30283 to 0.21064, saving model to ./artifacts/best_model_multiclass.keras
338/338 ━━━━━━━━━━━━━━━━━━━━ 9s 25ms/step - accuracy: 0.8473 - loss: 0.5354 - val_accuracy: 0.9565 - val_loss: 0.2106 - learni

In [30]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np




In [31]:
best_model = tf.keras.models.load_model(CKPT_PATH)


In [32]:
y_prob = best_model.predict(X_test_cnn)   # (N, 7)


676/676 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step


In [33]:
y_pred = y_prob.argmax(axis=1)             # (N,)


In [34]:
y_true = y_test_cnn

In [35]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true, y_pred)
print(cm)


[[ 394    0    0    0    0    0    0    0]
 [   0 2754    0    6    0    0    2    5]
 [   0    0 5998    2    0    0    0    0]
 [   0   28    0 5967    0    0    0    5]
 [   0    0    0    0    2    0    0    0]
 [   1    0    1    0    0    5    0    0]
 [   2    0    3    4    0    0 5990    1]
 [   0   30    0    5    0    0    0  401]]


In [36]:
from sklearn.metrics import classification_report

print(classification_report(
    y_true,
    y_pred,
    target_names=le_attack.classes_,
    digits=4
))


              precision    recall  f1-score   support

         bot     0.9924    1.0000    0.9962       394
  bruteforce     0.9794    0.9953    0.9873      2767
        ddos     0.9993    0.9997    0.9995      6000
         dos     0.9972    0.9945    0.9958      6000
  heartbleed     1.0000    1.0000    1.0000         2
infiltration     1.0000    0.7143    0.8333         7
    portscan     0.9997    0.9983    0.9990      6000
   webattack     0.9733    0.9197    0.9458       436

    accuracy                         0.9956     21606
   macro avg     0.9927    0.9527    0.9696     21606
weighted avg     0.9956    0.9956    0.9956     21606

